<a href="https://colab.research.google.com/github/DmitryKutsev/DeepHW/blob/master/my_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install youtokentome

In [16]:
!pip install alive-progress

In [17]:
# !wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
# !dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
# !apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
# !apt-get update
# !apt-get install cuda

In [18]:
# !pip install mxnet-cu92

In [106]:
import math
import numpy as np
import pandas as pd
from torchtext.data import Field, BucketIterator, TabularDataset
import random
import json
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [20]:
print(torch.cuda.device_count())

1


In [21]:
print(torch.cuda.get_device_name(0))

Tesla P4


In [22]:
device = torch.device('cuda:0')

In [23]:
# !pip install --upgrade --force-reinstall -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# !unzip '/content/drive/My Drive/unsupervised.csv.zip'

In [26]:
!unzip '/content/drive/My Drive/qa_data.jsonl.zip'

Archive:  /content/drive/My Drive/qa_data.jsonl.zip
replace qa_data.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [27]:
# questions = pd.read_csv('unsupervised.csv')
# questions.question = questions.question.map(lambda x: x.lower())

qa_data = list()

with open('qa_data.jsonl') as file_object:
    for line in file_object:
        qa_data.append(json.loads(line.strip()))
file_object.close()
qa_data = qa_data[:math.ceil(len(qa_data)*0.4)]#!!!!

In [28]:
with open('for_bpe.txt', 'w', encoding='utf-8') as f:
    for que in qa_data:
        f.write(que['question'] + '\n')
f.close()

vocab_size = 30_000
model_path = 'pretrained_bpe_lm.model'

In [29]:
%%time
# обучаем
# раскомментируйте этот код, чтобы обучить bpe
yttm.BPE.train(data='for_bpe.txt', vocab_size=vocab_size, model=model_path)

CPU times: user 9.94 s, sys: 909 ms, total: 10.8 s
Wall time: 6.63 s


In [30]:
# загружаем токенизатор
tokenizer = yttm.BPE(model=model_path)

In [31]:
qa_data = qa_data[:math.ceil(len(qa_data)*0.6)]

In [37]:
my_pairs = []
max_length = 32
for i in qa_data:
  quest = i['question']
  for resp in i['responses']:
    pair = (quest, resp)
    # pair.append(tok_quest, tokenizer.encode(resp, bos=True, eos=True))
    my_pairs.append(pair)

In [38]:
# tok_pairs = []
# max_length = 32
# for i in qa_data:
  
#   tok_quest = tokenizer.encode(i['question'], eos=True)
#   for resp in i['responses']:
#     pair = (tok_quest, tokenizer.encode(resp, eos=True))
#     # pair.append(tok_quest, tokenizer.encode(resp, bos=True, eos=True))
#     tok_pairs.append(pair)


In [41]:
import pandas as pd 

In [45]:
my_frame = pd.DataFrame(my_pairs)

In [113]:
my_frame.columns = ['q', 'ans']
my_frame = my_frame[:4000]
my_frame

,q,ans
0,долго ли идут деньги с яндексденег на карту visa?,нет. прорыв 35 ;)
1,можно ли зарегистрировать авто в другом регионе,можно на родственника из того региона.. . а п...
2,что делать если у меня очень тонкие ногти а хо...,витамины и умная эмаль (каждый день)
3,что делать если у меня очень тонкие ногти а хо...,ванночки с морской солью. с вечера мажь ногти ...
4,что делать если у меня очень тонкие ногти а хо...,"умная эмаль, витамины, йод, и поменьше крась л..."
...,...,...
3995,моя дочь хочет в художественную гимнастику ей ...,нет! поздно! отдайте в бальные танцы.
3996,какие лучше купить витамины для 17-летней дево...,"витамины фирм ""амвей"", ""арго"""
3997,какие лучше купить витамины для 17-летней дево...,алфавит - ничего лучше раздельного приёма ещё ...
3998,что вы хотели бы забыть ?,забыть что старость приближаетца


In [114]:
my_frame.to_csv('my_frame.csv', index=False)  

In [115]:
max_length = 20
def tokenize(text):
  return tokenizer.encode(text, output_type=yttm.OutputType.SUBWORD)[:max_length]

In [116]:
SRC = Field(tokenize=tokenize, init_token='<sos>', eos_token='<eos>')
TRG = Field(tokenize=tokenize, init_token='<sos>', eos_token='<eos>')

In [117]:
dataset = TabularDataset(path='my_frame.csv', 
                         format='csv', 
                         fields=[ ('ans', TRG), ('q', SRC),],
                         skip_header=True)

In [118]:
train_data, valid_data, test_data = dataset.split(split_ratio=[0.7, 0.1, 0.2], 
                                            random_state=random.getstate())

In [119]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [120]:
print(vars(train_data.examples[0])['ans'])

['▁как', '▁уговорить', '▁родителей', '▁купить', '▁мопе', 'д?']


In [121]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 256

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     sort_key=lambda x: len(x.ans), 
     sort_within_batch=False,
     device = device)

In [122]:
print (len(SRC.vocab), len(TRG.vocab))
print (SRC.vocab.freqs.most_common(10))
print (TRG.vocab.freqs.most_common(10))

4087 4662
[('▁не', 763), ('▁и', 724), ('▁в', 564), ('.', 515), ('▁а', 390), ('▁на', 365), (',', 269), ('!', 269), ('▁что', 267), ('▁это', 255)]
[('▁в', 647), ('▁как', 499), ('▁на', 495), ('▁что', 401), ('▁и', 401), ('▁не', 367), ('▁а', 362), ('▁с', 324), ('▁вы', 305), ('▁ли', 233)]


In [123]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        embedded = self.dropout(self.embedding(src))        
        outputs, (hidden, cell) = self.rnn(embedded)

        
        return hidden, cell

In [124]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        embedded = self.dropout(self.embedding(src))        
        outputs, (hidden, cell) = self.rnn(embedded)

        
        return hidden, cell

In [125]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        prediction = self.fc_out(output.squeeze(0))
        
        return prediction, hidden, cell

In [126]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)
        input = trg[0,:]
        
        for t in range(1, trg_len):

            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1) 
            input = trg[t] if teacher_force else top1
        
        return outputs

In [127]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.6
DEC_DROPOUT = 0.6

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [128]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(4087, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.6)
    (dropout): Dropout(p=0.6, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(4662, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.6)
    (fc_out): Linear(in_features=512, out_features=4662, bias=True)
    (dropout): Dropout(p=0.6, inplace=False)
  )
)

In [129]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,987,766 trainable parameters


In [130]:
optimizer = optim.Adam(model.parameters())

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
pad_idx = TRG.vocab.stoi['<pad>']
print(TRG.pad_token)  
print(TRG.vocab.stoi[TRG.pad_token]) 

criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

<pad>
1


In [131]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    my_losses = []
    for i, batch in enumerate(iterator):
        # progress_bar = tqdm(total=len(iterator), desc=f'{ i }')
        src = batch.q
        trg = batch.ans
        
        optimizer.zero_grad()
        
        output = model(src, trg)

        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        my_losses.append(loss.item())
        if i%10 == 0:
          print(f'fmean losses: { np.mean(my_losses[-1000:]) } ', f'iter { i }' )
        # progress_bar.set_postfix(loss=np.mean(my_losses[-1000:]),
                            # perplexity=np.exp(np.mean(my_losses[-1000:])))
        # progress_bar.update()
     # progress_bar.close()
    return epoch_loss / len(iterator)

In [132]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.q
            trg = batch.ans

            output = model(src, trg, 0) 
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [133]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [134]:
import time

In [135]:
N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    # print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

fmean losses: 8.465895652770996  iter 0
fmean losses: 7.713003548708829  iter 10
Epoch: 01 | Time: 0m 2s
	Train Loss: 7.713 | Train PPL: 2237.252


In [148]:
def translate_sentence(sentence,src_field,trg_field,model,device, max_len=50):
    model.eval()

    if isinstance(sentence,str):
        tokens = tokenizer.encode(sentence, output_type=yttm.OutputType.SUBWORD)
    else:
        tokens = tokenizer.encode(sentence, output_type=yttm.OutputType.SUBWORD)

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    #
    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    return trg_tokens[1:-1] # remove <sos> and <eos>

In [153]:
example_idx = 24

src = vars(train_data.examples[example_idx])['q']
src = " ".join(src)
trg = vars(train_data.examples[example_idx])['ans']

print(f'src = {src}')
print(f'trg = {trg}')

translation = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

src = ▁бро сь ▁его, ▁пока ▁не ▁поздно ! ▁его ▁не ▁передела ешь !!!
trg = ['▁как', '▁нужно', '▁поступить', '▁с', '▁мужчиной,', '▁который', '▁претен', 'дует', '▁на', '▁твоё', '▁личное', '▁простран', 'ство', '▁дик', 'ту', 'я', '▁свои', '▁условия?', ')))']
predicted trg = ['▁как', '▁в', '▁в']


In [151]:
translation = translate_sentence('что делать', SRC, TRG, model, device)

print(f'predicted trg = {translation}')

translation = translate_sentence('зачем мне это все?', SRC, TRG, model, device)

print(f'predicted trg = {translation}')

translation = translate_sentence('это не нормально, памагити', SRC, TRG, model, device)

print(f'predicted trg = {translation}')

translation = translate_sentence('зачем?', SRC, TRG, model, device)

print(f'predicted trg = {translation}')

translation = translate_sentence('что делать, если холодно', SRC, TRG, model, device)

print(f'predicted trg = {translation}')

predicted trg = ['▁как', '<unk>']
predicted trg = ['▁как', '▁в']
predicted trg = ['▁как', '▁в']
predicted trg = ['<unk>', '<unk>']
predicted trg = ['▁как', '<unk>']
